In [2]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error

# -----------------------------
# Load data
# -----------------------------
df = pd.read_csv("data.csv")

# -----------------------------
# Use push laps only
# -----------------------------
df = df[df["IsPushLap"] == 1].copy()

# -----------------------------
# Convert sector times to seconds
# -----------------------------
for col in ["Sector1Time", "Sector2Time", "Sector3Time"]:
    if col in df.columns:
        df[col] = pd.to_timedelta(df[col], errors="coerce").dt.total_seconds()

# -----------------------------
# Q3 ONLY (most stable & accurate)
# -----------------------------
df = df[df["QualiSegment"] == "Q3"].copy()

# -----------------------------
# Create SESSION MEDIAN (for delta target)
# -----------------------------
df["SessionMedianLap"] = (
    df.groupby(["Event", "Session", "QualiSegment"])["LapTime_sec"]
      .transform("median")
)

# -----------------------------
# NEW TARGET: DELTA LAP TIME
# -----------------------------
df["LapDelta_sec"] = df["LapTime_sec"] - df["SessionMedianLap"]
TARGET = "LapDelta_sec"

# -----------------------------
# Driver–Track interactions (HUGE gain)
# -----------------------------
df["Driver_Track"] = df["Driver"] + "_" + df["CircuitName"]
df["Team_Track"]   = df["Team"] + "_" + df["CircuitName"]

# -----------------------------
# Feature selection
# -----------------------------
categorical_features = [
    "Driver", "Team", "Compound", "Event", "Session",
    "QualiSegment", "CircuitName", "Country",
    "TrackType", "LapSpeedClass",
    "Driver_Track", "Team_Track"
]

numeric_features = [
    "TyreLife", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST",
    "TrackLength_m", "NumCorners", "CornerDensity",
    "AvgCornerSpacing_m", "AirTemp", "TrackTemp",
    "WindSpeed", "Altitude_m", "DRSZones"
]

features = categorical_features + numeric_features

X = df[features]
y = df[TARGET]

# -----------------------------
# Time-based split (no leakage)
# -----------------------------
train_idx = df["Year"] <= 2022
test_idx  = df["Year"] >= 2023

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

train_pool = Pool(
    X_train, y_train,
    cat_features=categorical_features
)

test_pool = Pool(
    X_test, y_test,
    cat_features=categorical_features
)

# -----------------------------
# Model (tuned for quali)
# -----------------------------
model = CatBoostRegressor(
    iterations=2500,
    learning_rate=0.03,
    depth=6,
    loss_function="MAE",
    eval_metric="MAE",
    random_seed=42,
    early_stopping_rounds=200,
    verbose=200
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    train_pool,
    eval_set=test_pool,
    use_best_model=True
)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f"\n🔥 Q3 DELTA MAE: {mae:.3f} seconds")

# -----------------------------
# Save model
# -----------------------------
model.save_model("quali_q3_delta_model.cbm")
print("✅ Model saved as quali_q3_delta_model.cbm")


0:	learn: 8.7973504	test: 8.0618593	best: 8.0618593 (0)	total: 127ms	remaining: 5m 17s
200:	learn: 3.3377612	test: 4.1390354	best: 4.1337358 (168)	total: 16.8s	remaining: 3m 11s
400:	learn: 2.7795394	test: 4.1167977	best: 4.0776031 (382)	total: 34.3s	remaining: 2m 59s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 4.077603121
bestIteration = 382

Shrink model to first 383 iterations.

🔥 Q3 DELTA MAE: 4.078 seconds
✅ Model saved as quali_q3_delta_model.cbm
